In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [366]:
from music21 import *
import os.path
import glob
import subprocess
import json
import copy

import sys
sys.path.append('/home/fl350/bachbot/scripts')
import quiz

Harmonizations have obvious repeating 16th notes: this fixes them

In [360]:
def fix_repeating_semiquavers(s):
    new_score = copy.deepcopy(s)
    for m in new_score.parts[0].getElementsByClass('Measure'):
        for start in np.arange(0.0, 4.0, 0.5):
            chords = m.notesAndRests.getElementsByOffset(start, start+0.5, includeEndBoundary=False)
            if len(chords) > 1:
                prev_el = chords[0]
                el = chords[1]
                prev_el.removeRedundantPitches(inPlace=True)
                el.removeRedundantPitches(inPlace=True)

                for prev_n, n in zip(sorted(prev_el._notes), sorted(el._notes)):
                    if prev_n.pitch == n.pitch:
                        if not prev_n.tie or prev_n.tie.type == 'stop':
                            prev_n.tie = tie.Tie('start')
                            n.tie = tie.Tie('stop')
    return new_score

In [364]:
EXPERIMENT_DIR = '/home/fl350/8-1-bachbot-questions'

In [ ]:
xml_files = [q['generated'][:-3]+'xml'
     for qg in json.load(open(EXPERIMENT_DIR + '/questions.json', 'r')).values()
     for q in qg
     if q['generated'][:3] == 'BWV']
for xml_file in xml_files:
    fp = '/home/fl350/bachbot/scratch/harm_out/{}'.format(xml_file)
    s = converter.parseFile(fp)
    fix_repeating_semiquavers(s).write('xml', EXPERIMENT_DIR + '/{}'.format(xml_file))
    quiz.xml_to_mp3(
        EXPERIMENT_DIR + '/{}'.format(xml_file),
        EXPERIMENT_DIR
    )
    print 'Processed: ' + xml_file

Processed: BWV-268-mask-Alto-fermatas.xml
Processed: BWV-438-mask-Alto-fermatas.xml
Processed: BWV-27.6-mask-Bass-fermatas.xml
Processed: BWV-293-mask-Bass-fermatas.xml
Processed: BWV-168.6-mask-Alto-Tenor-Bass-fermatas.xml
Processed: BWV-270-mask-Alto-Tenor-Bass-fermatas.xml
Processed: BWV-248.5-mask-Alto-Tenor-Bass-fermatas.xml
